In [1]:
from sklearn.linear_model import LinearRegression
import joblib
from google.cloud import aiplatform
from google.cloud import storage
import sklearn
import sys

In [2]:
GCS_BUCKET_NAME = "PLACEHOLDER_YOUR_BUCKET_NAME" #Multi region bucket that encompasses both regions
GCS_PREFIX = "vertex-load-balancing/test-models" 
# The full directory path (Vertex expects the DIRECTORY, not the file path)
ARTIFACT_URI = f"gs://{GCS_BUCKET_NAME}/{GCS_PREFIX}"

PROJECT_ID = "PLACEHOLDER_YOUR_PRODUCER_PROJECT" # Update this
REGION1 = "us-central1"
REGION2 = "us-east4"

In [4]:
def create_and_upload_model(model_name):
    print(f"Using scikit-learn version: {sklearn.__version__}")
    print(f"Using Python version: {sys.version}")

    print("1. Creating dummy model locally...")
    model = LinearRegression()
    model.fit([[1], [2]], [1, 2])

    local_filename = "model.joblib"
    joblib.dump(model, local_filename)

    print(f"2. Uploading {local_filename} to {ARTIFACT_URI}...")
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(GCS_BUCKET_NAME)
    blob = bucket.blob(f"{GCS_PREFIX}/{local_filename}")
    blob.upload_from_filename(local_filename)
    print("   Upload complete.")

    print("3. Registering model in Vertex AI...")
    # USE A NEWER CONTAINER IMAGE
    new_container_uri = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-5:latest"
    print(f"Using container image: {new_container_uri}")

    model_ver = aiplatform.Model.upload(
        display_name=model_name,
        artifact_uri=ARTIFACT_URI, # Points to the folder containing model.joblib
        serving_container_image_uri=new_container_uri, # UPDATED
        sync=True
    )

    print(f"✅ Success! Model ID: {model_ver.name}")
    return model_ver.name

**Upload the model to `REGION1` and take note of the model ID**

In [5]:
aiplatform.init(project=PROJECT_ID, location=REGION1)

In [8]:
create_and_upload_model(f"sklearn-dummy-model-{REGION1}")

Using scikit-learn version: 1.7.2
Using Python version: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:29:10) [GCC 14.3.0]
1. Creating dummy model locally...
2. Uploading model.joblib to gs://premium-support-tse-xpn-host-us/bernieongewe/test-models...
   Upload complete.
3. Registering model in Vertex AI...
Using container image: us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-5:latest
Creating Model
Create Model backing LRO: projects/277465195603/locations/us-central1/models/8476098854641467392/operations/7515821459487326208
Model created. Resource name: projects/277465195603/locations/us-central1/models/8476098854641467392@1
To use this Model in another session:
model = aiplatform.Model('projects/277465195603/locations/us-central1/models/8476098854641467392@1')
✅ Success! Model ID: 8476098854641467392


'8476098854641467392'

**Now upload the model to `REGION2` and take note of the second model ID**

In [9]:
aiplatform.init(project=PROJECT_ID, location=REGION2)

In [10]:
create_and_upload_model(f"sklearn-dummy-model-{REGION2}")

Using scikit-learn version: 1.7.2
Using Python version: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:29:10) [GCC 14.3.0]
1. Creating dummy model locally...
2. Uploading model.joblib to gs://premium-support-tse-xpn-host-us/bernieongewe/test-models...
   Upload complete.
3. Registering model in Vertex AI...
Using container image: us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-5:latest
Creating Model
Create Model backing LRO: projects/277465195603/locations/us-east4/models/8883337070848770048/operations/6944024234213507072
Model created. Resource name: projects/277465195603/locations/us-east4/models/8883337070848770048@1
To use this Model in another session:
model = aiplatform.Model('projects/277465195603/locations/us-east4/models/8883337070848770048@1')
✅ Success! Model ID: 8883337070848770048


'8883337070848770048'